# Transfer learning tests
Eamon Whalen

In [8]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet
from pointRegressorSurrogateModel import PointRegressor

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load pre-training data

In [ ]:
doeFiles = np.sort(glob.glob("/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/*1000.csv"))
for doeFile in doeFiles:
    designName = doeFile.split('/')[-1].split('_N')[0]
    print(f'loading {designName}')
    allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)
    allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
    trainSets[designName] = trainData

## 2. Load test data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
testGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max()
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Initial training

## 4. Transfer learning study

In [20]:
saveDir = './results/transferLrn01/'
dataDir = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/"
trainDataFiles = glob.glob(os.path.join(dataDir, 'design_7*.csv'))
trainDataFiles.remove(doeFile)

allResults = []
for trainDataFile in trainDataFiles:
    trainData = loadGhGraphs(trainDataFile, NUM_DV=5)
    trainSize = len(trainData)
    print(f'loaded train set of size {trainSize}')
    
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                     flatten=False, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir=saveDir+f'{trainSize}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['set'] = 'Train'
    trainRes['train size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['set'] = 'Test'
    testRes['train size'] = trainSize
    allResults.append(testRes)
    
pd.DataFrame(allResults)

30
trained 30 random forest models in 3.63 seconds
10
trained 30 random forest models in 3.40 seconds


,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,set,train size
0,0.000171,0.004807,0.110205,0.945597,0.953342,0.907048,0.788241,Train,30
1,0.000301,0.007306,0.239722,-6.456106,0.566105,-1.629988,-15.108212,Test,30
2,0.000590,0.009096,0.163803,0.866579,0.922484,0.845893,0.788937,Train,10
3,0.000219,0.009061,0.306289,-2.643653,0.394297,-1.249129,-13.832980,Test,10
